In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

In [2]:
def get_basic_info(key):
    key1 = key.split("/")[0]
    if "ARCTIC" in key1:
        data_type = "Ground Truth"
        speaker = key.split("/")[1]
        accent = spk2acc[speaker]
    elif "PD-AST"==key1:
        data_type = "SPAccenT"
        speaker, accent = key.split("/")[1:]
    else:
        data_type = "AC_"
        if "smaller" in key1:
            data_type += "small"
        elif "small" in key1:
            data_type += "normal"
        else:
            data_type += "large"
        if "80mel" in key1:
            data_type += "_BigVGAN"
        elif "hifiganmel" in key1:
            data_type += "_HiFiGAN"
        else:
            data_type += "_Vocos"
        if "VCTK" in key1:
            data_type += "_VCTK"
        speaker = "SLT"
        accent = mode
    return data_type, speaker, accent

spk2acc = {
    "SLT": "English",
    "ASI": "Hindi",
    "TNI": "Hindi",
    "HKK": "Korean",
    "YDCK": "Korean",
}

In [3]:
mode = "Hindi"
hindilabels = [
    "CMU-ARCTIC/SLT",
    "L2-ARCTIC/ASI",
    "L2-ARCTIC/TNI",
    "PD-AST/SLT/English",
    "PD-AST/SLT/Hindi",
    "PD-AST/ASI/Hindi",
    "PD-AST/TNI/Hindi",
    
    # "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating_small/100000",
    # "VTN_fine-tuning_nocondition_gt2syn_melmel_hubert_norepeating/100000",
    # "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating_small/100000",
    # "VTN_fine-tuning_nocondition_mix2synVCTK3hr_melmel_hubert_norepeating/100000",

    # "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating_small/100000",
    # "VTN_fine-tuning_nocondition_gt2syn_80mel80mel_hubert_norepeating/100000",
    # "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating_small/100000",
    # "VTN_fine-tuning_nocondition_mix2synVCTK3hr_80mel80mel_hubert_norepeating/100000",
    
    "VTN_fine-tuning_nocondition_gt2syn_hifiganmelhifiganmel_hubert_norepeating/100000",
    "VTN_fine-tuning_nocondition_mix2synVCTK3hr_hifiganmelhifiganmel_hubert_norepeating/100000",
]

koreanlabels = [
    "L2-ARCTIC/HKK",
    "L2-ARCTIC/YDCK",
    "PD-AST/SLT/English",
    "PD-AST/SLT/Korean",
    "PD-AST/HKK/Korean",
    "PD-AST/YDCK/Korean",
]
if mode=="Hindi":
    labels = hindilabels
elif mode=="Korean":
    labels = koreanlabels

In [4]:
save_dir = f"evaluation/accent/"

new_columns = []
for cl in ["Data Type", "Speaker", "Accent"]:
    new_columns += [("basics", cl)]
for cl in ["classification score (before softmax)", "classification prob.", "AECS ratio"]:
    new_columns += [("accent", cl)]
new_columns = pd.MultiIndex.from_tuples(new_columns)

basics = []
scores = []
for dirname in labels:
    data_type, speaker, accent = get_basic_info(dirname)
    path = save_dir + f'{"___".join(dirname.split("/"))}.npy'
    raws, probs, embs = np.load(path)
    scores += [[raws.mean(), probs.mean(), embs.mean()]]
    basics += [[data_type, speaker, accent]]
    
array = np.concatenate([basics, scores], axis=1)
accentdf = pd.DataFrame(array, columns=new_columns)
accentdf.loc[:, "accent"] =  np.round(accentdf.loc[:, "accent"].values.astype(float), 4)
accentdf

basics                   \
               Data Type Speaker   Accent   
0           Ground Truth     SLT  English   
1           Ground Truth     ASI    Hindi   
2           Ground Truth     TNI    Hindi   
3               SPAccenT     SLT  English   
4               SPAccenT     SLT    Hindi   
5               SPAccenT     ASI    Hindi   
6               SPAccenT     TNI    Hindi   
7       AC_large_HiFiGAN     SLT    Hindi   
8  AC_large_HiFiGAN_VCTK     SLT    Hindi   

                                 accent                                  
  classification score (before softmax) classification prob. AECS ratio  
0                               -9.2517               0.0000    -0.4621  
1                               13.3049               0.6320     0.2239  
2                               17.5397               0.8838     0.2867  
3                               -9.0417               0.0000    -0.5686  
4                               13.9551               0.8193     0.5686  
5                               11.8960               0.3538     0.2373  
6                               14.6826               0.7237     0.2771  
7                               12.6856               0.8008     0.4109  
8                               15.3088               0.8972     0.4646

In [5]:
save_dir = f"evaluation/wers/individual/"

new_columns = []
for cl in ["Data Type", "Speaker", "Accent"]:
    new_columns += [("basics", cl)]
for cl in ["tiny", "base", "small", "medium", "large"]:
    new_columns += [("WER", cl)]
new_columns = pd.MultiIndex.from_tuples(new_columns)

basics = []
wers = []
cers = []
for dirname in hindilabels + koreanlabels:
    data_type, speaker, accent = get_basic_info(dirname)
    path = save_dir + f'{"___".join(dirname.split("/"))}.npy'
    wer_list, cer_list = np.load(path)
    wers += [list(wer_list)]
    cers += [list(cer_list)]
    basics += [[data_type, speaker, accent]]
    
new_columns = []
for cl in ["Data Type", "Speaker", "Accent"]:
    new_columns += [("basics", cl)]
for cl in ["tiny", "base", "small", "medium", "large"]:
    new_columns += [("WER", cl)]
new_columns = pd.MultiIndex.from_tuples(new_columns)
array = np.concatenate([basics, wers], axis=1)
werdf = pd.DataFrame(array, columns=new_columns)
werdf.loc[:, "WER"] =  werdf.loc[:, "WER"].values.astype(float)

new_columns = []
for cl in ["Data Type", "Speaker", "Accent"]:
    new_columns += [("basics", cl)]
for cl in ["tiny", "base", "small", "medium", "large"]:
    new_columns += [("CER", cl)]
new_columns = pd.MultiIndex.from_tuples(new_columns)
array = np.concatenate([basics, cers], axis=1)
cerdf = pd.DataFrame(array, columns=new_columns)
cerdf.loc[:, "CER"] =  cerdf.loc[:, "CER"].values.astype(float)
werdf

basics                        WER                      \
                Data Type Speaker   Accent      tiny      base     small   
0            Ground Truth     SLT  English  0.059016  0.033880  0.025137   
1            Ground Truth     ASI    Hindi  0.145355  0.093989  0.067760   
2            Ground Truth     TNI    Hindi  0.191257  0.125683  0.077596   
3                SPAccenT     SLT  English  0.036066  0.026230  0.021858   
4                SPAccenT     SLT    Hindi  0.203279  0.149727  0.113661   
5                SPAccenT     ASI    Hindi  0.158470  0.123497  0.092896   
6                SPAccenT     TNI    Hindi  0.157377  0.104918  0.085246   
7        AC_large_HiFiGAN     SLT    Hindi  0.283060  0.253552  0.220765   
8   AC_large_HiFiGAN_VCTK     SLT    Hindi  0.239344  0.195628  0.166120   
9            Ground Truth     HKK   Korean  0.239344  0.196721  0.143169   
10           Ground Truth    YDCK   Korean  0.143169  0.122404  0.084153   
11               SPAccenT     SLT  English  0.036066  0.026230  0.021858   
12               SPAccenT     SLT   Korean  0.884153  0.734426  0.522404   
13               SPAccenT     HKK   Korean  0.890710  0.651366  0.425137   
14               SPAccenT    YDCK   Korean  0.850273  0.718033  0.515847   

                        
      medium     large  
0   0.025137  0.019672  
1   0.062295  0.046995  
2   0.067760  0.057923  
3   0.028415  0.017486  
4   0.110383  0.085246  
5   0.090710  0.093989  
6   0.084153  0.076503  
7   0.181421  0.139891  
8   0.131148  0.087432  
9   0.134426  0.125683  
10  0.075410  0.051366  
11  0.028415  0.017486  
12  0.469945  0.506011  
13  0.375956  0.617486  
14  0.459016  0.426230

In [6]:
result = pd.concat([werdf[["basics", "WER"]],  cerdf[["CER"]], accentdf[["accent"]]], axis=1)
result.iloc[0:6, -1:] = np.nan
result

basics                        WER                      \
                Data Type Speaker   Accent      tiny      base     small   
0            Ground Truth     SLT  English  0.059016  0.033880  0.025137   
1            Ground Truth     ASI    Hindi  0.145355  0.093989  0.067760   
2            Ground Truth     TNI    Hindi  0.191257  0.125683  0.077596   
3                SPAccenT     SLT  English  0.036066  0.026230  0.021858   
4                SPAccenT     SLT    Hindi  0.203279  0.149727  0.113661   
5                SPAccenT     ASI    Hindi  0.158470  0.123497  0.092896   
6                SPAccenT     TNI    Hindi  0.157377  0.104918  0.085246   
7        AC_large_HiFiGAN     SLT    Hindi  0.283060  0.253552  0.220765   
8   AC_large_HiFiGAN_VCTK     SLT    Hindi  0.239344  0.195628  0.166120   
9            Ground Truth     HKK   Korean  0.239344  0.196721  0.143169   
10           Ground Truth    YDCK   Korean  0.143169  0.122404  0.084153   
11               SPAccenT     SLT  English  0.036066  0.026230  0.021858   
12               SPAccenT     SLT   Korean  0.884153  0.734426  0.522404   
13               SPAccenT     HKK   Korean  0.890710  0.651366  0.425137   
14               SPAccenT    YDCK   Korean  0.850273  0.718033  0.515847   

                             CER                                          \
      medium     large      tiny      base     small    medium     large   
0   0.025137  0.019672  0.023256  0.011420  0.007475  0.007475  0.005814   
1   0.062295  0.046995  0.058970  0.037168  0.025125  0.025332  0.016196   
2   0.067760  0.057923  0.099252  0.060839  0.036752  0.030939  0.027409   
3   0.028415  0.017486  0.011836  0.007683  0.006437  0.008098  0.004568   
4   0.110383  0.085246  0.093023  0.062708  0.046719  0.046096  0.036545   
5   0.090710  0.093989  0.072051  0.055440  0.039037  0.039452  0.043812   
6   0.084153  0.076503  0.066445  0.045473  0.038206  0.033430  0.032392   
7   0.181421  0.139891  0.150332  0.140158  0.114618  0.100706  0.081188   
8   0.131148  0.087432  0.129983  0.103613  0.088455  0.071429  0.045266   
9   0.134426  0.125683  0.118563  0.094269  0.070806  0.063746  0.068729   
10  0.075410  0.051366  0.065615  0.053987  0.035922  0.030108  0.019311   
11  0.028415  0.017486  0.011836  0.007683  0.006437  0.008098  0.004568   
12  0.469945  0.506011  0.570183  0.449751  0.298796  0.280108  0.456811   
13  0.375956  0.617486  0.576620  0.385590  0.240656  0.204527  0.583264   
14  0.459016  0.426230  0.526993  0.441653  0.299003  0.260590  0.382683   

                                  accent                                  
   classification score (before softmax) classification prob. AECS ratio  
0                                -9.2517               0.0000        NaN  
1                                13.3049               0.6320        NaN  
2                                17.5397               0.8838        NaN  
3                                -9.0417               0.0000        NaN  
4                                13.9551               0.8193        NaN  
5                                11.8960               0.3538        NaN  
6                                14.6826               0.7237     0.2771  
7                                12.6856               0.8008     0.4109  
8                                15.3088               0.8972     0.4646  
9                                    NaN                  NaN        NaN  
10                                   NaN                  NaN        NaN  
11                                   NaN                  NaN        NaN  
12                                   NaN                  NaN        NaN  
13                                   NaN                  NaN        NaN  
14                                   NaN                  NaN        NaN

In [7]:
result.iloc[[0,4,7,8,9,10,11,12], [0,1,2,7,12,14,15]]
# result.iloc[:-6,[0,1,2,7,12,14,15]]

basics                        WER       CER  \
                Data Type Speaker   Accent     large     large   
0            Ground Truth     SLT  English  0.019672  0.005814   
4                SPAccenT     SLT    Hindi  0.085246  0.036545   
7        AC_large_HiFiGAN     SLT    Hindi  0.139891  0.081188   
8   AC_large_HiFiGAN_VCTK     SLT    Hindi  0.087432  0.045266   
9            Ground Truth     HKK   Korean  0.125683  0.068729   
10           Ground Truth    YDCK   Korean  0.051366  0.019311   
11               SPAccenT     SLT  English  0.017486  0.004568   
12               SPAccenT     SLT   Korean  0.506011  0.456811   

                 accent             
   classification prob. AECS ratio  
0                0.0000        NaN  
4                0.8193        NaN  
7                0.8008     0.4109  
8                0.8972     0.4646  
9                   NaN        NaN  
10                  NaN        NaN  
11                  NaN        NaN  
12                  NaN        NaN

In [8]:
import math
def get_latex_from_pandas(meandf, columns, indices, title="Title", label="Label", itvldf=None, multicolumns={}, multiindices={}, midrules=[], hdashedlines=[], pm_list=[], baseround_list=[], comparison=[], highestmin=[], twocolumn=False):
    if len(baseround_list)==0:
        baseround_list = [3]*len(columns)

    indexnum = 1 + int(len(multiindices))
    bestdf = meandf.copy()
    bestdf[:] = 0
    for com in comparison:
        a = meandf.loc[np.array(list(indices.keys()))[com], list(columns.keys())]
        a.iloc[:, highestmin] = -a.iloc[:, highestmin]
        a = a==a.max()
        for key in a.index:
            bestdf.loc[key] = a.loc[key].astype(float)

    if twocolumn:
        print("\\begin{table*}[!h]")
    else:
        print("\\begin{table}[!h]")
    print("\caption{" + title + "}")
    print("\label{table:" + label + "}")
    print("\\centering")
    cl_def = ""
    cl_def += "l"*(indexnum)
    cl_def += "c"*len(columns)

    text = "\\begin{tabular}{" + cl_def + "}"
    print(text)
    print("\\toprule")
    if len(multicolumns)>0:
        print(" & ".join([""]*(indexnum) + [f"\multicolumn{{{multicolumns[key]}}}{{c}}{{{key}}}" for key in multicolumns]) + "\\\\")
        linetxt = ""
        start = indexnum+1
        for key in multicolumns:
            end = start+multicolumns[key]-1
            linetxt += f"\\cmidrule(lr){{{start}-{end}}}"
            start = end+1
        print(linetxt)
    print(f" & ".join([""]*(indexnum+1) + list(columns.values()))[2:] + "\\\\")
    print(f"\midrule")
    for i in range(len(indices)):
        key = list(indices.keys())[i]
        keyname = indices[key]
        start = f"{keyname} &"
        vlist = []
        for k in range(len(columns)):
            base_round = baseround_list[k]
            valuekey = list(columns.keys())[k]
            v = meandf.loc[key, valuekey]
            if str(v)=="nan":
                vstr = "-"
            else:
                if v>0:
                    val = f"{np.round(v,base_round)}".ljust(base_round+2, '0')
                elif v==0:
                    val = "0."+"0"*base_round
                else:
                    val = f"{np.round(v,base_round)}".ljust(base_round+2, '0')
                    pass
                vstr = str(val)
                if bestdf.loc[key, valuekey]==1.0:
                    vstr = f"\\textbf{{{vstr}}}"
            if k in pm_list:
                ivl = itvldf.loc[key, valuekey]
                ivl = f"{np.round(ivl,base_round)}".ljust(base_round+2, '0')
                vstr = vstr + f"{{\\tiny $\\pm$ {{{ivl}}} }}"
            vlist += [vstr]
        print(start + " & ".join(vlist) + "\\\\")
        if i+1 in midrules:
            print("\midrule")
        if i+1 in hdashedlines:
            print("\\addlinespace[0.1em]\\hdashline\\addlinespace[0.3em]")
    print("\\bottomrule")
    print("\end{tabular}")
    if twocolumn:
        print("\end{table*}")
    else:
        print("\end{table}")

In [62]:
ah = pd.read_csv("evaluation/MUSHRA/ACTHindi.csv", index_col=0)
ak = pd.read_csv("evaluation/MUSHRA/ACTKorean.csv", index_col=0)
nh = pd.read_csv("evaluation/MUSHRA/NATHindi.csv", index_col=0)
nk = pd.read_csv("evaluation/MUSHRA/NATKorean.csv", index_col=0)

ah.index = ["---".join(get_basic_info("/".join(a.split("___")))) for a in list(ah.index)]
ak.index = ["---".join(get_basic_info("/".join(a.split("___")))) for a in list(ak.index)]
ak.index = [a+"---Korean" if "---SLT---English" in a else a for a in ak.index]

amushra = pd.concat([ah, ak], axis=0)

In [ ]:
pd.concat([df, mushra, index=df.index, columns=["Speech-MUSHRA", "Accent-MUSHRA"])], axis=1)

In [87]:
amushra

,mean,interval
Ground Truth---SLT---English,11.111111,7.422941
SPAccenT---SLT---English,8.736842,2.850744
Ground Truth---ASI---Hindi,59.235294,12.314906
Ground Truth---TNI---Hindi,79.764706,4.194873
SPAccenT---SLT---Hindi,57.666667,10.824045
SPAccenT---ASI---Hindi,75.526316,5.526776
SPAccenT---TNI---Hindi,84.722222,3.732316
AC_large_HiFiGAN---SLT---Hindi,36.166667,8.233795
AC_large_HiFiGAN_VCTK---SLT---Hindi,47.125000,10.946183
Ground Truth---SLT---English---Korean,5.375000,0.622019


In [90]:
mushra

,Speech-MUSHRA,Accent-MUSHRA
Ground Truth---SLT---English,7.422941,7.422941
Ground Truth---ASI---Hindi,12.314906,12.314906
Ground Truth---TNI---Hindi,4.194873,4.194873
SPAccenT---SLT---English,2.850744,2.850744
SPAccenT---SLT---Hindi,10.824045,10.824045
SPAccenT---ASI---Hindi,5.526776,5.526776
SPAccenT---TNI---Hindi,3.732316,3.732316
AC_large_HiFiGAN---SLT---Hindi,8.233795,8.233795
AC_large_HiFiGAN_VCTK---SLT---Hindi,10.946183,10.946183
Ground Truth---HKK---Korean,10.875891,10.875891


In [91]:
amushra

,mean,interval
Ground Truth---SLT---English,11.111111,7.422941
SPAccenT---SLT---English,8.736842,2.850744
Ground Truth---ASI---Hindi,59.235294,12.314906
Ground Truth---TNI---Hindi,79.764706,4.194873
SPAccenT---SLT---Hindi,57.666667,10.824045
SPAccenT---ASI---Hindi,75.526316,5.526776
SPAccenT---TNI---Hindi,84.722222,3.732316
AC_large_HiFiGAN---SLT---Hindi,36.166667,8.233795
AC_large_HiFiGAN_VCTK---SLT---Hindi,47.125000,10.946183
Ground Truth---SLT---English---Korean,5.375000,0.622019


In [92]:
mushra

,Speech-MUSHRA,Accent-MUSHRA
Ground Truth---SLT---English,7.422941,7.422941
Ground Truth---ASI---Hindi,12.314906,12.314906
Ground Truth---TNI---Hindi,4.194873,4.194873
SPAccenT---SLT---English,2.850744,2.850744
SPAccenT---SLT---Hindi,10.824045,10.824045
SPAccenT---ASI---Hindi,5.526776,5.526776
SPAccenT---TNI---Hindi,3.732316,3.732316
AC_large_HiFiGAN---SLT---Hindi,8.233795,8.233795
AC_large_HiFiGAN_VCTK---SLT---Hindi,10.946183,10.946183
Ground Truth---HKK---Korean,10.875891,10.875891


In [96]:
amushra.shape

(16, 2)

In [112]:
df = result.iloc[:,3:]
df.index = ["---".join(a) for a in basics]
df = df[[("WER", "large"), ("CER", "large"), ("accent", "classification prob."), ("accent", "AECS ratio")]]
df.columns = ["WER", "CER", "accent prob.", "AECS ratio"]
df = df[~df.index.duplicated(keep='first')]
# Add MUSHRA results
mushra = pd.DataFrame(pd.concat([amushra[["mean"]], amushra.loc[amushra.index][["mean"]]], axis=1).values, index=amushra.index, columns=["Speech-MUSHRA", "Accent-MUSHRA"])
meandf = pd.concat([df, mushra], axis=1)
mushra = pd.DataFrame(pd.concat([amushra[["interval"]], amushra.loc[amushra.index][["interval"]]], axis=1).values, index=amushra.index, columns=["Speech-MUSHRA", "Accent-MUSHRA"])
itvldf = pd.concat([df, mushra], axis=1)
# meandf = pd.concat([df, pd.DataFrame(50*np.ones((len(df), 2)), index=df.index, columns=["Speech-MUSHRA", "Accent-MUSHRA"])], axis=1)
# itvldf = pd.concat([df, pd.DataFrame(0.1**np.ones((len(df), 2)), index=df.index, columns=["Speech-MUSHRA", "Accent-MUSHRA"])], axis=1)
# meandf = pd.concat([meandf, pd.DataFrame(np.array([np.nan, np.nan, np.nan, np.nan, 20.0, 20.0]).reshape(1, -1), index=["Ground Truth---SLT---English---Korean"], columns=meandf.columns)])
# meandf = pd.concat([meandf, pd.DataFrame(np.array([np.nan, np.nan, np.nan, np.nan, 20.0, 20.0]).reshape(1, -1), index=["SPAccenT---SLT---English---Korean"], columns=meandf.columns)])
# itvldf = pd.concat([itvldf, pd.DataFrame(np.array([np.nan, np.nan, np.nan, np.nan, 0.2, 0.2]).reshape(1, -1), index=["Ground Truth---SLT---English---Korean"], columns=meandf.columns)])
# itvldf = pd.concat([itvldf, pd.DataFrame(np.array([np.nan, np.nan, np.nan, np.nan, 0.2, 0.2]).reshape(1, -1), index=["SPAccenT---SLT---English---Korean"], columns=meandf.columns)])

title = "Evaluation Results for Accent Voice Conversion"
label = "result_conversion"
columns = {
    "Speech-MUSHRA": "MUSURA ($\\uparrow$)",
    "WER": "WER ($\\downarrow$)",
    "CER": "CER ($\\downarrow$)",
    "Accent-MUSHRA": "MUSURA ($\\uparrow$)",
    "accent prob.": "Classification Prob. ($\\uparrow$)",
    "AECS ratio": "AECS Diff. ($\\uparrow$)",
}
indices = {
    "Ground Truth---SLT---English": "Ground Truth (American)",
    "SPAccenT---SLT---English": "SPAccenT (American)",
    "SPAccenT---SLT---Hindi": "SPAccenT (Hindi)",
    "AC_large_HiFiGAN---SLT---Hindi": "w/o Data Augmentation",
    "AC_large_HiFiGAN_VCTK---SLT---Hindi": "w/ Data Augmentation (ours)",
}
multicolumns = { # {}
    "Speech Quality": 3,
    "Accentedness": 3,
}
multiindices = {} # {}
midrules = [3] # []
hdashedlines = [] # []
pm_list = [0, 3] # []
baseround_list = [2, 3, 3, 2, 3, 3]
comparison = [ # []
    [3,4],
]
highestmin = [1, 2] # []
twocolumn = True # False

get_latex_from_pandas(meandf, columns, indices, title, label, itvldf, multicolumns, multiindices, midrules, hdashedlines, pm_list, baseround_list, comparison, highestmin, twocolumn)

\begin{table*}[!h]
\caption{Evaluation Results for Accent Voice Conversion}
\label{table:result_conversion}
\centering
\begin{tabular}{lcccccc}
\toprule
 & \multicolumn{3}{c}{Speech Quality} & \multicolumn{3}{c}{Accentedness}\\
\cmidrule(lr){2-4}\cmidrule(lr){5-7}
  & MUSURA ($\uparrow$) & WER ($\downarrow$) & CER ($\downarrow$) & MUSURA ($\uparrow$) & Classification Prob. ($\uparrow$) & AECS Diff. ($\uparrow$)\\
\midrule
Ground Truth (American) &11.11{\tiny $\pm$ {7.42} } & 0.020 & 0.006 & 11.11{\tiny $\pm$ {7.42} } & 0.000 & -\\
SPAccenT (American) &8.74{\tiny $\pm$ {2.85} } & 0.017 & 0.005 & 8.74{\tiny $\pm$ {2.85} } & 0.000 & -\\
SPAccenT (Hindi) &57.67{\tiny $\pm$ {10.82} } & 0.085 & 0.037 & 57.67{\tiny $\pm$ {10.82} } & 0.819 & -\\
\midrule
w/o Data Augmentation &36.17{\tiny $\pm$ {8.23} } & 0.140 & 0.081 & 36.17{\tiny $\pm$ {8.23} } & 0.801 & 0.411\\
w/ Data Augmentation (ours) &\textbf{47.12}{\tiny $\pm$ {10.95} } & \textbf{0.087} & \textbf{0.045} & \textbf{47.12}{\tiny $\pm$ {

In [113]:
title = "MUSHRA results for Dataset Analysis: GT and ST represent Ground Truth and SPAccenT, respectively."
label = "result_analysis"
columns = {
    "Speech-MUSHRA": "Naturalness ($\\uparrow$)",
    "Accent-MUSHRA": "Accentedness ($\\uparrow$)",
}
indices = {
    "Ground Truth---SLT---English": "GT (SLT)",
    "SPAccenT---SLT---English": "ST (SLT/American)",
    "Ground Truth---ASI---Hindi": "GT (ASI)",
    "Ground Truth---TNI---Hindi": "GT (TNI)",
    "SPAccenT---SLT---Hindi": "ST (SLT/Hindi)",
    "SPAccenT---ASI---Hindi": "ST (ASI/Hindi)",
    "SPAccenT---TNI---Hindi": "ST (TNI/Hindi)",
    "Ground Truth---SLT---English---Korean": "GT (SLT)",
    "SPAccenT---SLT---English---Korean": "ST (SLT/American)",
    "Ground Truth---HKK---Korean": "GT (HKK)",
    "Ground Truth---YDCK---Korean": "GT (YDCK)",
    "SPAccenT---SLT---Korean": "ST (SLT/Korean)",
    "SPAccenT---HKK---Korean": "ST (HKK/Korean)",
    "SPAccenT---YDCK---Korean": "ST (YDCK/Korean)",
}
multicolumns = {}
multiindices = {} # {}
midrules = [7] # []
hdashedlines = [2, 9] # []
pm_list = [0, 1] # []
baseround_list = [2,2]
comparison = [ # []
]
highestmin = [1, 2] # []
twocolumn = False # False

get_latex_from_pandas(meandf, columns, indices, title, label, itvldf, multicolumns, multiindices, midrules, hdashedlines, pm_list, baseround_list, comparison, highestmin, twocolumn)

\begin{table}[!h]
\caption{MUSHRA results for Dataset Analysis: GT and ST represent Ground Truth and SPAccenT, respectively.}
\label{table:result_analysis}
\centering
\begin{tabular}{lcc}
\toprule
  & Naturalness ($\uparrow$) & Accentedness ($\uparrow$)\\
\midrule
GT (SLT) &11.11{\tiny $\pm$ {7.42} } & 11.11{\tiny $\pm$ {7.42} }\\
ST (SLT/American) &8.74{\tiny $\pm$ {2.85} } & 8.74{\tiny $\pm$ {2.85} }\\
\addlinespace[0.1em]\hdashline\addlinespace[0.3em]
GT (ASI) &59.24{\tiny $\pm$ {12.31} } & 59.24{\tiny $\pm$ {12.31} }\\
GT (TNI) &79.76{\tiny $\pm$ {4.19} } & 79.76{\tiny $\pm$ {4.19} }\\
ST (SLT/Hindi) &57.67{\tiny $\pm$ {10.82} } & 57.67{\tiny $\pm$ {10.82} }\\
ST (ASI/Hindi) &75.53{\tiny $\pm$ {5.53} } & 75.53{\tiny $\pm$ {5.53} }\\
ST (TNI/Hindi) &84.72{\tiny $\pm$ {3.73} } & 84.72{\tiny $\pm$ {3.73} }\\
\midrule
GT (SLT) &5.38{\tiny $\pm$ {0.62} } & 5.38{\tiny $\pm$ {0.62} }\\
ST (SLT/American) &12.8{\tiny $\pm$ {19.41} } & 12.8{\tiny $\pm$ {19.41} }\\
\addlinespace[0.1em]\hdashl